In [2]:
# !pip install tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import deque
import random
from qnetwork import DQNetwork
from dataset import CustomImageDataset
from agent import DQNAgent

In [3]:
def reading_csv(folder_path: str, file_path: str):
    df = pd.read_csv(file_path)
    df = shuffle(df, random_state=42)
    df["filepath"] = folder_path + df["image_id"]
    return df

In [4]:
df = pd.read_csv("../digit-recognizer/train.csv")

In [5]:
# df = reading_csv("../cassava-leaf-disease-classification/train_images/", "../cassava-leaf-disease-classification/train.csv")

In [6]:
def drop_some_labels(df, label):
    label_ = df[df['label'] == label]
    n_to_drop = label_.shape[0]//3
    index_to_drop = label_.sample(n_to_drop).index
    df = df.drop(index_to_drop)
    return df

In [7]:
df = drop_some_labels(df, 2)

In [8]:
dataset = CustomImageDataset(df.drop(columns=["label"]), df["label"], (28, 28), 32)

No of total samples in dataset and their distribution:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), array([4132, 4684, 2785, 4351, 4072, 3795, 4137, 4401, 4063, 4188],
      dtype=int64))

Reward for each class.
	- Class 0 : 0.301741
	- Class 1 : 0.266182
	- Class 2 : 0.447682
	- Class 3 : 0.286554
	- Class 4 : 0.306187
	- Class 5 : 0.328536
	- Class 6 : 0.301376
	- Class 7 : 0.283298
	- Class 8 : 0.306865
	- Class 9 : 0.297706


In [9]:
network = DQNetwork(state_size=(28, 28, 3), action_size=len(dataset.get_class_num()), gamma = 0.95, epsilon = 1.0, learning_rate = 0.001)

No of total samples in dataset and their distribution:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), array([4132, 4684, 2785, 4351, 4072, 3795, 4137, 4401, 4063, 4188],
      dtype=int64))
Num GPUs Available:  0


In [10]:
agent = DQNAgent(network, dataset, state_size=(28, 28, 3), action_size = len(dataset.get_class_num()), memory=deque(maxlen=2000), epsilon=1.0)

No of total samples in dataset and their distribution:  (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64), array([4132, 4684, 2785, 4351, 4072, 3795, 4137, 4401, 4063, 4188],
      dtype=int64))


In [11]:
agent.train(10)

TypeError: '_ParallelMapDataset' object is not subscriptable